# Getting Started with GeoAgent

This notebook demonstrates the core features of GeoAgent — an AI agent for geospatial data search, analysis, and visualization. It covers multiple data types, locations, time ranges, and analysis workflows.

In [ ]:
# %pip install geoagent

## Setup

In [ ]:
import geoagent

print(f"GeoAgent version: {geoagent.__version__}")

In [ ]:
from geoagent.core.llm import get_available_providers, check_api_keys

print("Available providers:", get_available_providers())
print("API key status:", check_api_keys())

In [ ]:
from geoagent import GeoAgent

# Use any available provider:
# agent = GeoAgent()  # auto-detect
# agent = GeoAgent(provider="openai", model="gpt-4.1")
# agent = GeoAgent(provider="anthropic", model="claude-sonnet-4-5-20250929")
# agent = GeoAgent(provider="google", model="gemini-2.5-flash")
agent = GeoAgent(provider="ollama", model="llama3.1")

---

## 1. Sentinel-2 Satellite Imagery

Search for Sentinel-2 images over a specific location and time range.

In [ ]:
result = agent.chat("Show Sentinel-2 imagery of San Francisco in July 2025")
print(
    f"Success: {result.success} | Items: {result.data.total_items} | Time: {result.execution_time:.1f}s"
)
result.map

In [ ]:
print(result.code)

### Sentinel-2 with cloud cover filter

In [ ]:
result = agent.chat(
    "Find Sentinel-2 images of Tokyo with less than 10% cloud cover in March 2024"
)
print(f"Success: {result.success} | Items: {result.data.total_items}")
if result.data.items:
    for item in result.data.items[:3]:
        cc = item["properties"].get("eo:cloud_cover", "N/A")
        print(f"  {item['id']}: cloud cover {cc}%")
result.map

---

## 2. NDVI Analysis

Compute real NDVI from Sentinel-2 red and NIR bands.

In [ ]:
result = agent.chat("Compute NDVI for Knoxville Tennessee in July 2024")
print(f"Success: {result.success} | Time: {result.execution_time:.1f}s")
if result.analysis:
    stats = result.analysis.result_data.get("ndvi_stats", {})
    if stats:
        print(f"NDVI mean: {stats['mean']:.3f}")
        print(f"NDVI range: [{stats['min']:.3f}, {stats['max']:.3f}]")
        print(f"Pixels analyzed: {stats['pixels']:,}")
result.map

In [ ]:
print(result.code)

### NDVI for a different location

In [ ]:
result = agent.chat("Show NDVI for the Amazon rainforest in August 2024")
print(
    f"Success: {result.success} | Items: {result.data.total_items if result.data else 0}"
)
if result.analysis:
    stats = result.analysis.result_data.get("ndvi_stats", {})
    if stats and stats.get("mean"):
        print(f"NDVI mean: {stats['mean']:.3f}")
result.map

---

## 3. Land Cover (ESRI 10m)

Visualize land use / land cover data from the ESRI 10m annual dataset (Planetary Computer `io-lulc-9-class` collection).

In [ ]:
result = agent.chat("Show land cover for Denver Colorado")
print(
    f"Success: {result.success} | Items: {result.data.total_items} | Time: {result.execution_time:.1f}s"
)
if result.data.items:
    print(f"Collection: {result.data.items[0].get('collection')}")
result.map

In [ ]:
print(result.code)

### Land cover for another location

In [ ]:
result = agent.chat("Show land cover for New York City")
print(f"Success: {result.success} | Items: {result.data.total_items}")
result.map

---

## 4. Digital Elevation Model (DEM)

Visualize elevation data from the Copernicus DEM 30m dataset (Planetary Computer `cop-dem-glo-30` collection).

In [ ]:
result = agent.chat("Show DEM for Grand Canyon")
print(
    f"Success: {result.success} | Items: {result.data.total_items} | Time: {result.execution_time:.1f}s"
)
if result.data.items:
    print(f"Collection: {result.data.items[0].get('collection')}")
result.map

In [ ]:
print(result.code)

### Elevation for another location

In [ ]:
result = agent.chat("Show elevation map of Mount Rainier")
print(f"Success: {result.success} | Items: {result.data.total_items}")
result.map

---

## 5. Landsat Imagery

Search for Landsat Collection 2 Level-2 data.

In [ ]:
result = agent.chat("Find Landsat images of Denver in June 2024")
print(
    f"Success: {result.success} | Items: {result.data.total_items} | Time: {result.execution_time:.1f}s"
)
if result.data.items:
    for item in result.data.items[:3]:
        cc = item["properties"].get("eo:cloud_cover", "N/A")
        print(f"  {item['id']}: cloud cover {cc}%")
result.map

In [ ]:
print(result.code)

---

## 6. Different Locations

GeoAgent automatically geocodes place names to bounding boxes.

In [ ]:
# African location
result = agent.chat("Show Sentinel-2 imagery of Lagos Nigeria in January 2024")
print(f"Lagos - Success: {result.success} | Items: {result.data.total_items}")
result.map

In [ ]:
# European location
result = agent.chat("Show land cover for Paris France")
print(f"Paris - Success: {result.success} | Items: {result.data.total_items}")
result.map

In [ ]:
# Asian location
result = agent.chat("Show DEM for Mount Fuji")
print(f"Mount Fuji - Success: {result.success} | Items: {result.data.total_items}")
result.map

---

## 7. Different Time Ranges

In [ ]:
# Winter
result = agent.chat("Show Sentinel-2 imagery of Chicago in December 2023")
print(
    f"Chicago Dec 2023 - Success: {result.success} | Items: {result.data.total_items}"
)
result.map

In [ ]:
# Summer
result = agent.chat("Show Sentinel-2 imagery of Chicago in July 2024")
print(
    f"Chicago Jul 2024 - Success: {result.success} | Items: {result.data.total_items}"
)
result.map

---

## 8. Data Search Only

Use `agent.search()` to search for data without analysis or visualization.

In [ ]:
data = agent.search(
    "Find Sentinel-2 images of Seattle in August 2024 with low cloud cover"
)
print(f"Found {data.total_items} items")
print(f"Search params: {data.search_query}")
for item in data.items[:5]:
    cc = item["properties"].get("eo:cloud_cover", "N/A")
    dt = item["properties"].get("datetime", "")
    print(f"  {item['id']}: {dt[:10]} cloud={cc}%")

---

## 9. STAC Catalog Registry

Browse and use pre-configured STAC catalog endpoints.

In [ ]:
from geoagent.catalogs.registry import CatalogRegistry

reg = CatalogRegistry()
for cat in reg.list_catalogs():
    auth = " (auth required)" if cat.requires_auth else ""
    print(f"{cat.name:25s} {cat.url}{auth}")

In [ ]:
# Direct STAC search via registry
client = reg.get_client("planetary_computer")

results = client.search(
    collections=["cop-dem-glo-30"],
    bbox=[-112.2, 36.0, -111.8, 36.3],  # Grand Canyon
    max_items=3,
)

items = list(results.items())
print(f"Found {len(items)} DEM tiles:")
for item in items:
    print(f"  {item.id}: bbox={item.bbox}")

---

## Summary

| Data Type | Collection | Example Query |
|-----------|-----------|---------------|
| Sentinel-2 Imagery | `sentinel-2-l2a` | "Show Sentinel-2 imagery of Tokyo in March 2024" |
| Landsat Imagery | `landsat-c2-l2` | "Find Landsat images of Denver in June 2024" |
| NDVI Analysis | `sentinel-2-l2a` | "Compute NDVI for Knoxville in July 2024" |
| Land Cover (ESRI) | `io-lulc-9-class` | "Show land cover for Denver" |
| DEM / Elevation | `cop-dem-glo-30` | "Show DEM for Grand Canyon" |